In [239]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import quandl
from scipy import stats
from sklearn.linear_model import LinearRegression

In [272]:
with open('../quandl_key.txt', 'r') as f:
    quandl.ApiConfig.api_key = f.read()

tickers = ['AAPL', 'MSFT', 'AMZN', 'F', 'AMD', 'FB', 'MTCH', 'SQ', 'TWTR', 'DPZ']
data = quandl.get_table('WIKI/PRICES', 
                        ticker=tickers,
                        qopts={'columns':['ticker', 'date', 'adj_close']},
                        date={'gte':'2014-1-1', 'lte':datetime.today()},
                        paginate=True
                       ).set_index('date')

data = data.pivot(columns='ticker', values='adj_close').groupby(pd.Grouper(freq="M")).last()
df_ret = data.pct_change(1).dropna()
# change day to 1'st to line up with FF data
df_ret.index = [pd.datetime(dt_.year, dt_.month, 1) for dt_ in df_ret.index]

# pfl rets for equal-weighted pfl
df_ret['PFL'] = sum([1/len(df_ret.columns) * df_ret[col] for col in df_ret.columns])

df_ret.head()

ticker,AAPL,AMD,AMZN,DPZ,F,FB,MSFT,TWTR,PFL
2014-02-01,0.057474,0.081633,0.009507,0.119671,0.028743,0.094135,0.019996,-0.148682,0.032810
2014-03-01,0.019953,0.080863,-0.071072,-0.023390,0.013645,-0.120070,0.069956,-0.150064,-0.022522
2014-04-01,0.099396,0.019950,-0.095833,-0.033649,0.043269,-0.007636,-0.014394,-0.164988,-0.019236
2014-05-01,0.078720,-0.022005,0.027686,-0.025948,0.017957,0.058883,0.020386,-0.167565,-0.001486
2014-06-01,0.027662,0.047500,0.039130,0.012272,0.048662,0.063033,0.018564,0.262947,0.064971


In [273]:
# download here: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
df_ff = pd.read_csv('./data/F-F_Research_Data_Factors.CSV', 
                    skiprows=3, 
                    index_col='Date',
                    parse_dates=True,
                    date_parser=lambda x: pd.datetime.strptime(x, '%Y%m'))
df_ff['MKT'] = df_ff['MKT_RF'] + df_ff['RF']
df_ff /= 100
df_ff.head()

,MKT_RF,SMB,HML,RF,MKT
Date,,,,,
1926-07-01,0.0296,-0.0230,-0.0287,0.0022,0.0318
1926-08-01,0.0264,-0.0140,0.0419,0.0025,0.0289
1926-09-01,0.0036,-0.0132,0.0001,0.0023,0.0059
1926-10-01,-0.0324,0.0004,0.0051,0.0032,-0.0292
1926-11-01,0.0253,-0.0020,-0.0035,0.0031,0.0284


In [274]:
df_spy = pd.read_csv('./data/^GSPC.csv',
                     usecols=['Date', 'Adj Close'],
                     index_col='Date',
                     parse_dates=True)
df_spy_ret = df_spy.pct_change(1).dropna()
df_spy_ret.columns = ['SPY']
df_spy_ret.head()

,SPY
Date,
2010-02-01,0.028514
2010-03-01,0.058796
2010-04-01,0.014759
2010-05-01,-0.081976
2010-06-01,-0.053882


In [275]:
df = df_spy_ret.join(df_ret, how='inner').join(df_ff, how='inner')
df = df[df_ret.index[-1] - relativedelta(months=36):] # 3-year beta
df.head()

,SPY,AAPL,AMD,AMZN,DPZ,F,FB,MSFT,TWTR,PFL,MKT_RF,SMB,HML,RF,MKT
2015-03-01,-0.017396,-0.031372,-0.138264,-0.021202,-0.006525,-0.012240,0.041092,-0.072862,0.041597,-0.024972,-0.0112,0.0302,-0.0046,0.0,-0.0112
2015-04-01,0.008521,0.005786,-0.156716,0.133512,0.072601,-0.011900,-0.041902,0.196409,-0.222045,-0.003032,0.0059,-0.0297,0.0186,0.0,0.0059
2015-05-01,0.010491,0.045312,0.008850,0.017663,0.007510,-0.039873,0.005332,-0.030318,-0.058778,-0.005538,0.0136,0.0087,-0.0137,0.0,0.0136
2015-06-01,-0.021012,-0.037266,0.052632,0.011323,0.046513,-0.010547,0.083028,-0.057832,-0.012272,0.009447,-0.0153,0.0283,-0.0079,0.0,-0.0153
2015-07-01,0.019742,-0.032888,-0.195833,0.235113,0.003880,-0.002248,0.096135,0.057758,-0.143843,0.002259,0.0154,-0.0414,-0.0412,0.0,0.0154


### CAPM

In [278]:
X = np.reshape(np.array(df['MKT_RF']), (df.shape[0], 1))

print('Stock betas\n---------------')
for col in df_ret:
    reg = LinearRegression().fit(X=X, y=df[col]) # could use 'SPY' instead
    print(col, round(reg.coef_[0], 3))

Stock betas
---------------
AAPL 1.077
AMD 3.422
AMZN 1.571
DPZ 0.143
F 0.851
FB 0.681
MSFT 1.208
TWTR 0.844
PFL 1.225


### FF

In [277]:
print('Stock | MFT_RF | SMB | HML\n---------------')
X = df[['MKT_RF', 'SMB', 'HML']]
for col in df_ret:
    reg = LinearRegression().fit(X=X, y=df[col]) # could use 'SPY' instead
    print(col, [round(coef, 3) for coef in reg.coef_])

Stock | MFT_RF | SMB | HML
---------------
AAPL [1.089, -0.247, -0.692]
AMD [3.415, 0.203, 1.218]
AMZN [1.622, -0.862, -1.057]
DPZ [0.149, -0.143, -0.532]
F [0.833, 0.292, 0.259]
FB [0.725, -0.724, -0.672]
MSFT [1.288, -1.237, 0.119]
TWTR [0.745, 1.415, -1.674]
PFL [1.233, -0.163, -0.379]
